# APEx STAC catalog integration with openEO

The project catalog provided by APEx can be used in combination with an openEO based platform.

Do note that openEO is an open standard, and not all platforms may support all use cases. Also the produced STAC metadata may have differences that are relevant to your project.
Therefore we always recommend to check the envisioned use cases with your provider of choice.

## Supported use cases

1. Scripted upload of openEO generated results into APEx catalog (supported by any openEO instance).
2. Direct publishing of results to object storage (requires support for openEO 'workspace' API).
3. Direct publishing of results to STAC catalogue (requires support for openEO 'workspace' API, and specifically merging into existing catalog.)

## Use case 1: Scripted upload

This use case is most broadly supported because it simply takes STAC metadata of openEO job results, downloads it, and then ingests it into a catalog.

The most important drawbacks of this method:

- Data is first downloaded from openEO backend and then uploaded again, which can take time and bandwidth, and may even be interrupted when facing network instability.
- Extra project specific code is needed to perform the task.



In [3]:
import openeo
from osgeo_utils.gdal2tiles import GlobalMercator

con = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [3]:

tilegrid = GlobalMercator()
bounds = tilegrid.TileLatLonBounds(233, 214, 9)
resolution = tilegrid.Resolution(12)
print(bounds)
print(resolution)


cube = con.load_collection(
    "SENTINEL2_L2A",
    bands=["B04", "B03", "B02"],
    temporal_extent="2019-08-19",
    spatial_extent={
        "west": bounds[1],
        "south": abs(bounds[2]),#abs is weird, does GlobalMercator return wrong latitude values?
        "east": bounds[3],
        "north": abs(bounds[0]),
    }
)
cube.result_node().update_arguments(featureflags={"tilesize": 256})#force block size in output tiff
result = cube.resample_spatial(resolution=resolution,projection="EPSG:3857")
job = result.execute_batch("gran_canaria.tiff",title="Gran Canaria",filename_prefix="gran_canaria")

(-28.30438068296276, -16.171874999999993, -27.68352808378777, -15.468750000000012)
38.21851414258813
Authenticated using refresh token.
0:00:00 Job 'j-241106f371e448088b97aeb8d0c9c19f': send 'start'
0:00:14 Job 'j-241106f371e448088b97aeb8d0c9c19f': created (progress 0%)
0:00:21 Job 'j-241106f371e448088b97aeb8d0c9c19f': created (progress 0%)
0:00:27 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:00:36 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:00:46 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:00:58 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:01:14 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:01:33 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:01:58 Job 'j-241106f371e448088b97aeb8d0c9c19f': running (progress N/A)
0:02:28 Job 'j-241106f371e448088b97aeb8d0c9c19f': finished (progress 100%)


In [4]:
job = con.job("j-241106f371e448088b97aeb8d0c9c19f")
job

<BatchJob job_id='j-241106f371e448088b97aeb8d0c9c19f'>

In [16]:
import pystac

stac_metadata_dict = job.get_results().get_metadata()

#remove collection assets, we will rely on item links
del stac_metadata_dict["assets"]
collection = pystac.Collection.from_dict(stac_metadata_dict)
collection

<Collection id=j-241106f371e448088b97aeb8d0c9c19f>

## Convert online openEO collection to local collection

This step cleans up links to avoid that they point to the openEO API, which in some cases requires authentication.

There's an [open issue](https://github.com/Open-EO/openeo-python-client/issues/184) to integrate this in the api.

In [18]:
#remove collection and canoncial links
collection.remove_links(rel="collection")
collection.remove_links(rel="canonical")

items = list(collection.get_stac_objects(rel=pystac.RelType.ITEM))
for i in items:
    i.remove_links(rel="collection")
    i.remove_links(rel="canonical")



collection.set_self_href("/tmp/collection.json")
collection.normalize_hrefs('/tmp/', skip_unresolved=True)

def asset_transform(name,a):
    a.href = "/tmp/" + name
    return a

#this step can transform asset hrefs as well, 
#c2=catalog.map_assets(asset_transform)



collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)
collection

<Collection id=j-241106f371e448088b97aeb8d0c9c19f>

In [14]:
list(collection.get_items())

[<Item id=gran_canaria_2019-08-19Z.tif>]

## Upload to STAC API

The collection metadata has now been cleaned up and can be added to the STAC API.
Here we create the full collection, but it's also possible to only add the generated item to a new collection.

At this point, it is also recommended to improve the metadata quality by adding additional metadata properties.

### Important note

At this point, the actual Geotiff is still located on the openEO backend, and can be accessed by any tool via the signed url.
This url will however expire, so for more permanent catalogs, the data file needs to be moved to a better location.
To do this, simply download the tiff file and upload it to an online location of your preference.


# Use case 2/3: direct upload by openEO

In this case, we will let openEO export the collection to an external storage.

Because this case depends on the availability of the "export_workspace" process, we check for its existence first.

In [20]:

con.describe_process("export_workspace")

{'categories': ['export', 'stac'],
 'description': 'Exports the given processing results made available through a STAC resource (e.g., a STAC Collection) to the given user workspace. The STAC resource itself is exported with all STAC resources and assets underneath.',
 'experimental': True,
 'id': 'export_workspace',
 'parameters': [{'description': 'The data to export to the user workspace as a STAC resource.',
   'name': 'data',
   'schema': {'subtype': 'stac', 'type': 'object'}},
  {'description': 'The identifier of the workspace to export to.',
   'name': 'workspace',
   'schema': {'pattern': '^[\\w\\-\\.~]+$',
    'subtype': 'workspace-id',
    'type': 'string'}},
  {'default': None,
   'description': 'Provides a cloud-specific path identifier to a STAC resource to merge the given STAC resource into. If not provided, the STAC resource is kept separate from any other STAC resources in the workspace.',
   'name': 'merge',
   'optional': True,
   'schema': {'type': ['string', 'null']}}],
 'returns': {'description': 'Returns the potentially updated STAC resource.',
  'schema': {'subtype': 'stac', 'type': 'object'}},
 'summary': 'Export data to a cloud user workspace'}

In [ ]:
exported_result = result.export_workspace("my_workspace_id", merge="my_stac_collection")

job = exported_result.execute_batch("gran_canaria.tiff",title="Gran Canaria",filename_prefix="gran_canaria")